In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn import preprocessing
from sklearn.model_selection import KFold

In [3]:
train_data = np.array(pd.read_csv('data/trainSet.csv'))
# test  = np.array(pd.read_csv('test.csv'))

# tra, val = train_test_split(train, train_size=0.8)
# print(tra.shape)
# print(val.shape)

In [4]:
kf = KFold(n_splits=5)

k_fold_data = []

for tra_idx, val_idx in kf.split(train_data):
    tra = train_data[tra_idx]
    val = train_data[val_idx]
    k_fold_data.append((tra, val))
    print(len(tra) + len(val))

200000
200000
200000
200000
200000


In [5]:
class TrainValDataset(Dataset):
    def __init__(self, data):
        self.X = preprocessing.normalize(data[:, :-1])
        self.Y = data[:, -1]
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return (self.X[idx, :], self.Y[idx])

In [6]:
class Net(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, dropout=0.5):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [7]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [8]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc  = 0
    
    model.train()
    
    for idx, (X, labels) in enumerate(iterator):
        
        X = X.float()
        labels = labels.float()
        labels = labels.view(-1, 1)
        
        optimizer.zero_grad()
        
        predictions = model(X)
        
        loss = criterion(predictions, labels)
        
        acc  = binary_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc  += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [9]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for idx, (X, labels) in enumerate(iterator):
            
            X = X.float()
            labels = labels.float()
            labels = labels.view(-1, 1)
            
            predictions = model(X)
            
            loss = criterion(predictions, labels)
            
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc  += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [10]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
def predict(model, iterator):
    
    model.eval()
    
    results = []
    
    with torch.no_grad():
    
        for idx, X in enumerate(iterator):
            
            X = X.float()
            
            predictions = model(X)
            
            round_pred = torch.round(torch.sigmoid(predictions))

            for i in range(predictions.size(0)):
                results.append(round_pred[i].item())
                
    return results

In [12]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.X = preprocessing.normalize(data)
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx, :]

In [17]:
import os

k_fold_results = []

test  = np.array(pd.read_csv('data/test set.csv'))

test_dataset = TestDataset(test)
test_loader  = DataLoader(
    test_dataset,
    batch_size=512,
    shuffle=False
)

for i in range(5):
    if not os.path.exists('result/{}'.format(i+1)):
        os.mkdir('result/{}'.format(i+1))
    
    (tra, val) = k_fold_data[i]
    
    train_dataset = TrainValDataset(tra)
    train_loader  = DataLoader(
        train_dataset,
        batch_size=512,
        shuffle=True
    )

    valid_dataset = TrainValDataset(val)
    valid_loader  = DataLoader(
        valid_dataset,
        batch_size=512,
        shuffle=False
    )
    
    INPUT_DIM  = 32
    HIDDEN_DIM = 128
    OUTPUT_DIM = 1
    DROPOUT    = 0.2

    model = Net(
        INPUT_DIM,
        HIDDEN_DIM,
        OUTPUT_DIM,
        DROPOUT
    )

    optimizer = optim.Adam(model.parameters())

    criterion = nn.BCEWithLogitsLoss()
    
    # training
    N_EPOCHS = 1000

    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):

        start_time = time.time()

        train_loss, train_acc = train(model, train_loader, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_loader, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'result/{}/nn_128_2.pt'.format(i+1))

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\tVal.  Loss: {valid_loss:.3f} | Val.  Acc: {valid_acc*100:.2f}%')
        
    # predicting
    model.load_state_dict(torch.load('result/{}/nn_128_2.pt'.format(i+1)))
    results = predict(model, test_loader)
    k_fold_results.append(results)

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.689 | Train Acc: 54.34%
	Val.  Loss: 0.716 | Val.  Acc: 39.02%
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.688 | Train Acc: 54.70%
	Val.  Loss: 0.720 | Val.  Acc: 39.02%
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 0.687 | Train Acc: 54.71%
	Val.  Loss: 0.715 | Val.  Acc: 39.03%
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 0.685 | Train Acc: 54.88%
	Val.  Loss: 0.717 | Val.  Acc: 39.37%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.683 | Train Acc: 55.26%
	Val.  Loss: 0.711 | Val.  Acc: 41.44%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.681 | Train Acc: 55.84%
	Val.  Loss: 0.705 | Val.  Acc: 45.05%
Epoch: 07 | Epoch Time: 0m 2s
	Train Loss: 0.679 | Train Acc: 56.36%
	Val.  Loss: 0.692 | Val.  Acc: 51.32%
Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 0.677 | Train Acc: 56.80%
	Val.  Loss: 0.691 | Val.  Acc: 51.76%
Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.675 | Train Acc: 57.11%
	Val.  Loss: 0.682 | Val.  Acc: 56.23%
Epoch: 10 | Epoch Time: 0m 2

Epoch: 77 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.61%
	Val.  Loss: 0.661 | Val.  Acc: 61.29%
Epoch: 78 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.64%
	Val.  Loss: 0.661 | Val.  Acc: 61.30%
Epoch: 79 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.61%
	Val.  Loss: 0.662 | Val.  Acc: 61.16%
Epoch: 80 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.63%
	Val.  Loss: 0.661 | Val.  Acc: 61.28%
Epoch: 81 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.58%
	Val.  Loss: 0.662 | Val.  Acc: 61.21%
Epoch: 82 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.55%
	Val.  Loss: 0.661 | Val.  Acc: 61.32%
Epoch: 83 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.59%
	Val.  Loss: 0.664 | Val.  Acc: 60.29%
Epoch: 84 | Epoch Time: 0m 3s
	Train Loss: 0.662 | Train Acc: 59.57%
	Val.  Loss: 0.664 | Val.  Acc: 60.35%
Epoch: 85 | Epoch Time: 0m 3s
	Train Loss: 0.663 | Train Acc: 59.65%
	Val.  Loss: 0.661 | Val.  Acc: 61.15%
Epoch: 86 | Epoch Time: 0m 3

Epoch: 153 | Epoch Time: 0m 3s
	Train Loss: 0.660 | Train Acc: 60.16%
	Val.  Loss: 0.657 | Val.  Acc: 61.86%
Epoch: 154 | Epoch Time: 0m 3s
	Train Loss: 0.660 | Train Acc: 60.18%
	Val.  Loss: 0.657 | Val.  Acc: 61.94%
Epoch: 155 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.29%
	Val.  Loss: 0.657 | Val.  Acc: 61.71%
Epoch: 156 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.16%
	Val.  Loss: 0.657 | Val.  Acc: 61.87%
Epoch: 157 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.36%
	Val.  Loss: 0.657 | Val.  Acc: 61.80%
Epoch: 158 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.20%
	Val.  Loss: 0.657 | Val.  Acc: 61.81%
Epoch: 159 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.21%
	Val.  Loss: 0.658 | Val.  Acc: 61.58%
Epoch: 160 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.21%
	Val.  Loss: 0.657 | Val.  Acc: 61.84%
Epoch: 161 | Epoch Time: 0m 3s
	Train Loss: 0.659 | Train Acc: 60.15%
	Val.  Loss: 0.657 | Val.  Acc: 61.60%
Epoch: 162 | Epoch 

Epoch: 229 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.41%
	Val.  Loss: 0.654 | Val.  Acc: 62.05%
Epoch: 230 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.45%
	Val.  Loss: 0.657 | Val.  Acc: 61.61%
Epoch: 231 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.49%
	Val.  Loss: 0.654 | Val.  Acc: 61.95%
Epoch: 232 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.64%
	Val.  Loss: 0.657 | Val.  Acc: 61.85%
Epoch: 233 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.51%
	Val.  Loss: 0.653 | Val.  Acc: 61.98%
Epoch: 234 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.47%
	Val.  Loss: 0.656 | Val.  Acc: 62.01%
Epoch: 235 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.53%
	Val.  Loss: 0.655 | Val.  Acc: 62.04%
Epoch: 236 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.49%
	Val.  Loss: 0.654 | Val.  Acc: 61.99%
Epoch: 237 | Epoch Time: 0m 3s
	Train Loss: 0.658 | Train Acc: 60.40%
	Val.  Loss: 0.656 | Val.  Acc: 61.56%
Epoch: 238 | Epoch 

Epoch: 305 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.55%
	Val.  Loss: 0.652 | Val.  Acc: 61.89%
Epoch: 306 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.67%
	Val.  Loss: 0.653 | Val.  Acc: 62.05%
Epoch: 307 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.65%
	Val.  Loss: 0.652 | Val.  Acc: 62.09%
Epoch: 308 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.71%
	Val.  Loss: 0.653 | Val.  Acc: 62.12%
Epoch: 309 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.62%
	Val.  Loss: 0.653 | Val.  Acc: 62.00%
Epoch: 310 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.67%
	Val.  Loss: 0.653 | Val.  Acc: 62.00%
Epoch: 311 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.56%
	Val.  Loss: 0.654 | Val.  Acc: 61.84%
Epoch: 312 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.65%
	Val.  Loss: 0.652 | Val.  Acc: 62.10%
Epoch: 313 | Epoch Time: 0m 3s
	Train Loss: 0.657 | Train Acc: 60.62%
	Val.  Loss: 0.654 | Val.  Acc: 62.11%
Epoch: 314 | Epoch 

...

Epoch: 637 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.23%
	Val.  Loss: 0.615 | Val.  Acc: 69.37%
Epoch: 638 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.27%
	Val.  Loss: 0.610 | Val.  Acc: 69.36%
Epoch: 639 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.31%
	Val.  Loss: 0.610 | Val.  Acc: 69.36%
Epoch: 640 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.16%
	Val.  Loss: 0.615 | Val.  Acc: 69.35%
Epoch: 641 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.12%
	Val.  Loss: 0.615 | Val.  Acc: 69.28%
Epoch: 642 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.22%
	Val.  Loss: 0.610 | Val.  Acc: 69.37%
Epoch: 643 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.24%
	Val.  Loss: 0.621 | Val.  Acc: 69.32%
Epoch: 644 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.35%
	Val.  Loss: 0.610 | Val.  Acc: 69.39%
Epoch: 645 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.33%
	Val.  Loss: 0.613 | Val.  Acc: 69.40%
Epoch: 646 | Epoch 

Epoch: 713 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.33%
	Val.  Loss: 0.614 | Val.  Acc: 69.45%
Epoch: 714 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.36%
	Val.  Loss: 0.611 | Val.  Acc: 69.42%
Epoch: 715 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.20%
	Val.  Loss: 0.612 | Val.  Acc: 69.35%
Epoch: 716 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.18%
	Val.  Loss: 0.614 | Val.  Acc: 69.34%
Epoch: 717 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.30%
	Val.  Loss: 0.613 | Val.  Acc: 69.33%
Epoch: 718 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.20%
	Val.  Loss: 0.614 | Val.  Acc: 69.35%
Epoch: 719 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.26%
	Val.  Loss: 0.615 | Val.  Acc: 69.42%
Epoch: 720 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.24%
	Val.  Loss: 0.611 | Val.  Acc: 69.33%
Epoch: 721 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.32%
	Val.  Loss: 0.612 | Val.  Acc: 69.40%
Epoch: 722 | Epoch 

Epoch: 789 | Epoch Time: 0m 2s
	Train Loss: 0.665 | Train Acc: 59.25%
	Val.  Loss: 0.611 | Val.  Acc: 69.38%
Epoch: 790 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.34%
	Val.  Loss: 0.610 | Val.  Acc: 69.40%
Epoch: 791 | Epoch Time: 0m 3s
	Train Loss: 0.664 | Train Acc: 59.29%
	Val.  Loss: 0.613 | Val.  Acc: 69.29%
Epoch: 792 | Epoch Time: 0m 3s
	Train Loss: 0.664 | Train Acc: 59.33%
	Val.  Loss: 0.612 | Val.  Acc: 69.42%
Epoch: 793 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.39%
	Val.  Loss: 0.614 | Val.  Acc: 69.25%
Epoch: 794 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.34%
	Val.  Loss: 0.613 | Val.  Acc: 69.39%
Epoch: 795 | Epoch Time: 0m 3s
	Train Loss: 0.665 | Train Acc: 59.24%
	Val.  Loss: 0.610 | Val.  Acc: 69.32%
Epoch: 796 | Epoch Time: 0m 3s
	Train Loss: 0.664 | Train Acc: 59.38%
	Val.  Loss: 0.614 | Val.  Acc: 69.38%
Epoch: 797 | Epoch Time: 0m 3s
	Train Loss: 0.664 | Train Acc: 59.35%
	Val.  Loss: 0.614 | Val.  Acc: 69.27%
Epoch: 798 | Epoch 

Epoch: 865 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.32%
	Val.  Loss: 0.613 | Val.  Acc: 69.40%
Epoch: 866 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.44%
	Val.  Loss: 0.609 | Val.  Acc: 69.36%
Epoch: 867 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.36%
	Val.  Loss: 0.610 | Val.  Acc: 69.31%
Epoch: 868 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.29%
	Val.  Loss: 0.615 | Val.  Acc: 69.27%
Epoch: 869 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.44%
	Val.  Loss: 0.611 | Val.  Acc: 69.35%
Epoch: 870 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.38%
	Val.  Loss: 0.614 | Val.  Acc: 69.35%
Epoch: 871 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.27%
	Val.  Loss: 0.613 | Val.  Acc: 69.40%
Epoch: 872 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.46%
	Val.  Loss: 0.611 | Val.  Acc: 69.48%
Epoch: 873 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.45%
	Val.  Loss: 0.611 | Val.  Acc: 69.35%
Epoch: 874 | Epoch 

Epoch: 941 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.31%
	Val.  Loss: 0.609 | Val.  Acc: 69.35%
Epoch: 942 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.45%
	Val.  Loss: 0.611 | Val.  Acc: 69.45%
Epoch: 943 | Epoch Time: 0m 3s
	Train Loss: 0.664 | Train Acc: 59.31%
	Val.  Loss: 0.612 | Val.  Acc: 69.39%
Epoch: 944 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.39%
	Val.  Loss: 0.613 | Val.  Acc: 69.41%
Epoch: 945 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.46%
	Val.  Loss: 0.611 | Val.  Acc: 69.37%
Epoch: 946 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.36%
	Val.  Loss: 0.613 | Val.  Acc: 69.44%
Epoch: 947 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.51%
	Val.  Loss: 0.614 | Val.  Acc: 69.33%
Epoch: 948 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.38%
	Val.  Loss: 0.611 | Val.  Acc: 69.36%
Epoch: 949 | Epoch Time: 0m 2s
	Train Loss: 0.664 | Train Acc: 59.36%
	Val.  Loss: 0.613 | Val.  Acc: 69.32%
Epoch: 950 | Epoch 

In [18]:
ensemble_result = []
for i in range(len(k_fold_results[0])):
    total = 0
    for k in range(5):
        total += k_fold_results[k][i]
    if total > 2:
        ensemble_result.append(1)
    else:
        ensemble_result.append(0)

In [19]:
len(ensemble_result)

183267

In [20]:
import pandas as pd

output = pd.DataFrame(data={'id': list(range(1, len(results)+1)), 'Predicted': ensemble_result})
output.to_csv('nn_5_fold.csv', index=False)